In [ ]:
import os
import argparse
import numpy as np
import pandas as pd
import sys
#sort the data with the help of users

In [ ]:
node_folder =  'wikidata/'
timestamp_col='revtime'
df_complete = pd.read_csv(node_folder + "wiki_consolidated.csv")
df_complete['revtime'] = pd.to_datetime(df_complete['revtime'])
df_complete

In [ ]:
#sort the # of users by distinct pages and get the top 200 users.
df_complete= df_complete[~df_complete['pagetitle'].str.contains("Wikipedia")]
print(df_complete['pagetitle'].nunique())
df_page_edits =  df_complete.groupby('username')['pagetitle'].agg(['nunique'])
df_page_edits = df_page_edits.sort_values(['nunique'], ascending=False, kind='mergesort')
user_list = list(df_page_edits.index.unique())
print(user_list[:200])

In [ ]:
import matplotlib.pyplot as plt
hist = df_page_edits.hist(column='nunique', bins=50)
df_page_edits.to_csv(node_folder + "wiki_num_users.csv")

In [ ]:
from numpy import genfromtxt
words = genfromtxt('wikidata/wiki_data_users.txt')
print(words)

In [ ]:
import pylab
pylab.rcParams['xtick.major.pad']='8'
pylab.rcParams['ytick.major.pad']='8'
#pylab.rcParams['font.sans-serif']='Arial'

from matplotlib import rc
rc('font', family='sans-serif')
rc('font', size=10.0)
rc('text', usetex=False)


from matplotlib.font_manager import FontProperties

panel_label_font = FontProperties().copy()
panel_label_font.set_weight("bold")
panel_label_font.set_size(12.0)
panel_label_font.set_family("sans-serif")

In [ ]:
%pylab inline
import powerlaw


data = words

fit = powerlaw.Fit(data, discrete=True)
#print(fit.)
####
figCCDF = fit.plot_pdf(color='r', linewidth=3, label="Pages edited count fit")
fit.power_law.plot_pdf(color='b', linestyle='--', linewidth=2,label="Power law fit",  ax=figCCDF)
#fit.plot_ccdf(color='r', linewidth=2, ax=figCCDF)
#fit.power_law.plot_ccdf(color='r', linestyle='--', ax=figCCDF)
####
figCCDF.set_ylabel(u"p(X)")
figCCDF.set_xlabel(r"count of distinct pages edited by user")
#handles, labels = fig.get_legend_handles_labels()
#figCCDF.legend(handles, labels, loc=2)

figname = 'FigCCDF'
savefig(figname+'.eps', bbox_inches='tight')

In [ ]:
#get all pages that have been edited by the users in user_list
min_list = user_list[:50]
df_limited_data = df_complete[df_complete.username.isin(min_list)]
df_limited_data

In [ ]:
pages_by_max_users = list(df_limited_data.pagetitle.unique())
print(len(pages_by_max_users))



In [ ]:
df_page_user = df_complete[df_complete.pagetitle.isin(pages_by_max_users)]
#df_temp = df_page_user.groupby('pagetitle')['username'].agg(['nunique'])
#df_temp = df_temp.sort_values(['nunique'], ascending=False, kind='mergesort')
df_page_user
print(len(df_page_user.username.unique()))
#df_temp

In [ ]:
df_with_user = df_page_user.groupby('pagetitle')['stiki_score'].agg(['mean', 'median'])

df_page_no_user = df_page_user[~df_page_user['username'].isin(min_list)]
df_page_no_user

In [ ]:
df_without_user = df_page_no_user.groupby('pagetitle')['stiki_score'].agg(['mean', 'median'])
df_without_user.rename(columns={'mean':'mean_without_user', 'median':'median_without_user'}, inplace=True)

df_final = df_with_user.merge(df_without_user, on='pagetitle', how='left')
df_final

In [ ]:
df_final.to_csv(node_folder + "wiki_hypothesis2.csv")